In [1]:
from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
sc.install_pypi_package("sklearn")
sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("numpy")
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
import pandas as pd
import numpy as np
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
10,application_1588472063188_0023,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processing /mnt/var/lib/livy/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.23.0-cp36-cp36m-manylinux1_x86_64.whl (7.3 MB)
  Using cached threadpoolctl-2.0.0-py3-none-any.whl (34 kB)
  Using cached scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl (26.1 MB)
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)

  Using cached pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl (10.5 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)


# Github Commands 

In [2]:
# !github ..... commands not working!!!

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Sample Data

In [ ]:
input_bucket = 's3://amazon-reviews-pds'
input_path = '/parquet/product_category=Books/*.parquet'
df = spark.read.parquet(input_bucket + input_path)

In [9]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|         US|   15444933|R1WWG70WK9VUCH|1848192576|     835940987|Standing Qigong f...|          5|            9|         10|   N|                Y|Informative AND i...|After attending a...| 2015-05-02|2015|
|         US|   20595117|R1EQ3POS0RIOD5|145162445X|     574044348|A Universe from N...|          4|            4|          7|   N|                N|Between 'Nothing'...

In [3]:
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(marketplace='US', customer_id='15444933', review_id='R1WWG70WK9VUCH', product_id='1848192576', product_parent='835940987', product_title='Standing Qigong for Health and Martial Arts - Zhan Zhuang', star_rating=5, helpful_votes=9, total_votes=10, vine='N', verified_purchase='Y', review_headline='Informative AND interesting!', review_body="After attending a few Qigong classes, I wanted to have a book to read and re-read the instructions so I could practice at home.  I also wanted to gain more of an understanding of the purpose and benefit of the movements in order to practice them with a more focused purpose.<br /><br />The book exceeded my expectations.  The explanations are very clear and are paired with photos showing the correct form.  The book itself is more than just the Qigong, it's a very interesting read.  I read the whole book in two days and will read it again. I rarely read books twice!  The book has provided the information and additional instruction that I was looking f

In [4]:
df.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, year: string]

In [6]:
df.count(
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20726160

# Import Real Data from S3  

In [7]:
# train_path =  's3://bosch-data-spark/data/train_numeric.csv'
# df_train_numeric = spark.read.load(train_path,format="csv",inferSchema = True,header=True)

# test_path =  's3://bosch-data-spark/data/test_numeric.csv'
# df_test_numeric = spark.read.load(test_path,format="csv",inferSchema = True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#parquet files already created

# df_train_numeric.write.parquet("s3://bosch-data-spark/data/df_train_numeric.parquet")
# df_test_numeric.write.parquet("s3://bosch-data-spark/data/df_test_numeric.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
dfs_train_numeric = spark.read.parquet("s3://bosch-data-spark/data/df_train_numeric.parquet")
dfs_test_numeric = spark.read.parquet("s3://bosch-data-spark/data/df_test_numeric.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
dfs_train_numeric.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1183747

In [6]:
len(dfs_train_numeric.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

970

In [11]:
dfs_train_numeric.count()

dfs_train_numeric.select(dfs_train_numeric[-1]).distinct().count()

# Categorical Features

In [12]:
# path =  's3://bosch-data-spark/data/train_categorical.csv'
# df_categorical_train = spark.read.load(path,format="csv",header=True)

# path = 's3://bosch-data-spark/data/test_categorical.csv'
# df_categorical_test = spark.read.load(path,format="csv",header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# df_categorical_train.write.parquet("s3://bosch-data-spark/data/df_train_categorical.parquet")
# df_categorical_test.write.parquet("s3://bosch-data-spark/data/df_test_categorical.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
dfs_train_categorical = spark.read.parquet("s3://bosch-data-spark/data/df_train_categorical.parquet")
dfs_test_categorical = spark.read.parquet("s3://bosch-data-spark/data/df_test_categorical.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_train_categorical.limit(100).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     Id L0_S1_F25 L0_S1_F27  ... L3_S49_F4237 L3_S49_F4239 L3_S49_F4240
0     4      None      None  ...         None         None         None
1     6      None      None  ...         None         None         None
2     7      None      None  ...         None         None         None
3     9      None      None  ...         None         None         None
4    11      None      None  ...         None         None         None
..  ...       ...       ...  ...          ...          ...          ...
95  192      None      None  ...         None         None         None
96  197      None      None  ...         None         None         None
97  200      None      None  ...         None         None         None
98  201      None      None  ...         None         None         None
99  205      None      None  ...         None         None         None

[100 rows x 2141 columns]

# Start of numerical feature exploration

In [16]:
sdf_train.select(sdf_train[0]).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+
|    Id|
+------+
|594807|
|594808|
|594810|
|594821|
|594824|
|594832|
|594833|
|594834|
|594838|
|594839|
+------+
only showing top 10 rows

In [17]:
sdf_test.select(sdf_train[0]).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+
|    Id|
+------+
|892308|
|892328|
|892332|
|892337|
|892347|
|892359|
|892374|
|892379|
|892386|
|892387|
+------+
only showing top 10 rows

In [9]:
df_train_categorical.select(df_train_categorical[0]).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'df_train_categorical' is not defined
Traceback (most recent call last):
NameError: name 'df_train_categorical' is not defined



# NOTE: On Join for PySpark, indicate the column match as a string rather than the commented out statement 

In [14]:
# inner_join = df_train_numeric.join(df_train_categorical, df_train_numeric.Id == df_train_categorical.Id )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# On Exploring the Categorical Dataset, it appears there are so many NaN's that it simply doesn't make sense to analyze it

In [4]:
inner_join_train = dfs_train_categorical.join(dfs_train_numeric, "Id" )
inner_join_test = dfs_test_categorical.join(dfs_test_numeric, "Id" )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
len(inner_join_train.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3110

In [117]:
def col_len(dfs):
    return len(dfs.columns)

def get_cols(dfs):
    return dfs.columns

def row_len(dfs):
    return df.count()

col_len(dfs_train_categorical) + col_len(dfs_train_numeric) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3111

In [116]:
inner_join_train.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1183747

In [123]:
inner_join[inner_join.columns[1:-1]].na.drop(how="all").rows

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'DataFrame' object has no attribute 'rows'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1301, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'rows'



In [124]:
inner_join.na.drop(thresh=3).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1183165

### Sample DF to test  

In [223]:
df = spark.createDataFrame(
    [(None, float('nan'), float('nan')), (1, float('nan'), float(5)), (None, None, np.nan), (1, float('nan'), float('nan')), (1, float('nan'), float(10)), (1, float('nan'), float('nan')), (1,float('nan'), float('nan'))],
    ('session', "timestamp1", "id2"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [195]:
df = spark.createDataFrame([
    (1, "B", "X1"), (None, None, None), (None, float('nan'), "X1"), (None, "C", None)],
    ("ID", "TYPE", "CODE")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [224]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+----+
|session|timestamp1| id2|
+-------+----------+----+
|   null|       NaN| NaN|
|      1|       NaN| 5.0|
|   null|      null| NaN|
|      1|       NaN| NaN|
|      1|       NaN|10.0|
|      1|       NaN| NaN|
|      1|       NaN| NaN|
+-------+----------+----+

In [226]:
df.na.drop(thresh=1).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+----+
|session|timestamp1| id2|
+-------+----------+----+
|      1|       NaN| 5.0|
|      1|       NaN| NaN|
|      1|       NaN|10.0|
|      1|       NaN| NaN|
|      1|       NaN| NaN|
+-------+----------+----+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+----+
|session|timestamp1| id2|
+-------+----------+----+
|   null|         1|null|
|      1|         2| 5.0|
|      1|         3| NaN|
|      1|         4|null|
|      1|         5|10.0|
|      1|         6| NaN|
|      1|         6| NaN|
+-------+----------+----+

In [162]:
drop_null_columns(df, threshold=4).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+
|session|timestamp1|
+-------+----------+
|      1|         1|
|      1|         2|
|      1|         3|
|      1|         4|
|      1|         5|
|      1|         6|
|      1|         6|
+-------+----------+

In [163]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

7

### Function To Drop all Columns containing Nan or Null 

In [165]:
threshold = inner_join_train.count() - 1 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [166]:
def drop_null_columns(df, threshold=threshold):
    """
    This function drops all columns which contain null values.
    :param df: A PySpark DataFrame
    """
    null_counts = df.select([F.count(F.when(isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v > threshold]
    df = df.drop(*to_drop)
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [167]:
len(inner_join.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3110

In [168]:
len(drop_null_columns(inner_join).columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2947

In [33]:
get_cols(dfs_train_categorical)[::col_len(dfs_train_categorical)-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Id', 'L3_S49_F4240']

In [35]:
get_cols(dfs_train_numeric)[::col_len(dfs_train_numeric)-1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Id', 'Response']

In [40]:
# get_cols(inner_join)[::col_len(dfs_train_categorical)-1]

get_cols(inner_join)[col_len(dfs_train_categorical):][:10]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F8', 'L0_S0_F10', 'L0_S0_F12', 'L0_S0_F14', 'L0_S0_F16', 'L0_S0_F18']

In [31]:
inner_join_train.createOrReplaceTempView("parquetFile")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
#correct query to 

from pyspark.sql.functions import isnan, when, count, col

inner_join.select([count(when(isnan(c), c)).alias(c) for c in inner_join.columns]).show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [17]:
# count all nan values for all columns
# inner_join.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in inner_join.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Start of Numerical / Categorical Exploration 

In [5]:
sdf_train,sdf_test = inner_join_train.randomSplit([0.8, 0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
inner_join_train.createOrReplaceTempView("parquetFile")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Revealed too many nulls, but good practice utilizing SQL

In [10]:
for column in dfs_train_categorical.columns[600:650]:
    print(sqlContext.sql(f"SELECT count({column}) as non_nulls,{column} FROM parquetFile where L0_S1_F25 is not null group by {column}").show())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-10:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1497



+---------+------------+
|non_nulls|L1_S24_F1160|
+---------+------------+
|        0|        null|
+---------+------------+

None
+---------+------------+
|non_nulls|L1_S24_F1162|
+---------+------------+
|        0|        null|
+---------+------------+

None
+---------+------------+
|non_nulls|L1_S24_F1164|
+---------+------------+
|        0|        null|
+---------+------------+

None
+---------+------------+
|non_nulls|L1_S24_F1165|
+---------+------------+
|        0|        null|
+---------+------------+

None
+---------+------------+
|non_nulls|L1_S24_F1167|
+---------+------------+
|        0|        null|
+---------+------------+

None
+---------+------------+
|non_nulls|L1_S24_F1169|
+---------+------------+
|        0|        null|
+---------+------------+

None
+---------+------------+
|non_nulls|L1_S24_F1177|
+---------+------------+
|        0|        null|
+---------+------------+

None
+---------+------------+
|non_nulls|L1_S24_F1179|
+---------+------------+
|       

In [22]:
# sdf_train_cum 

# sdf_train.columns[0:-1]

dfs_train_categorical[dfs_train_categorical.columns[:20]].show(1000)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|     Id|L0_S1_F25|L0_S1_F27|L0_S1_F29|L0_S1_F31|L0_S2_F33|L0_S2_F35|L0_S2_F37|L0_S2_F39|L0_S2_F41|L0_S2_F43|L0_S2_F45|L0_S2_F47|L0_S2_F49|L0_S2_F51|L0_S2_F53|L0_S2_F55|L0_S2_F57|L0_S2_F59|L0_S2_F61|
+-------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|1543132|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|
|1543134|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|     null|


In [20]:
dfs_train_numeric.columns[:20]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Id', 'L0_S0_F0', 'L0_S0_F2', 'L0_S0_F4', 'L0_S0_F6', 'L0_S0_F8', 'L0_S0_F10', 'L0_S0_F12', 'L0_S0_F14', 'L0_S0_F16', 'L0_S0_F18', 'L0_S0_F20', 'L0_S0_F22', 'L0_S1_F24', 'L0_S1_F28', 'L0_S2_F32', 'L0_S2_F36', 'L0_S2_F40', 'L0_S2_F44', 'L0_S2_F48', 'L0_S2_F52', 'L0_S2_F56', 'L0_S2_F60', 'L0_S2_F64', 'L0_S3_F68', 'L0_S3_F72', 'L0_S3_F76', 'L0_S3_F80', 'L0_S3_F84', 'L0_S3_F88', 'L0_S3_F92', 'L0_S3_F96', 'L0_S3_F100', 'L0_S4_F104', 'L0_S4_F109', 'L0_S5_F114', 'L0_S5_F116', 'L0_S6_F118', 'L0_S6_F122', 'L0_S6_F132', 'L0_S7_F136', 'L0_S7_F138', 'L0_S7_F142', 'L0_S8_F144', 'L0_S8_F146', 'L0_S8_F149', 'L0_S9_F155', 'L0_S9_F160', 'L0_S9_F165', 'L0_S9_F170']

In [109]:
sdf_test.head(1)[0][:100]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('1001139', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)

In [234]:
#zero null valyes in label column

sdf_train.select([count(when(isnan("Response"),True))]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

### Condensing the response 0 values down a bit 

In [3]:
dfs_train_numeric.createOrReplaceTempView("table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sqlContext.sql("select response,count(*) from table group by response").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+
|response|count(1)|
+--------+--------+
|       1|    6879|
|       0| 1176868|
+--------+--------+

In [5]:
filtered_dfs_train = sqlContext.sql("""(select * from table where response = 1)
                                       union      
                                       (select * from table where response = 0 limit 10000)""")



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sdf_train,sdf_test = filtered_dfs_train.randomSplit([0.8, 0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
input_cols_wrangle = sdf_train.columns[1:-1]


def process(df):
    # row_count for threshold below
    col_thresh = df.count() - 1 
    
    #Remove Columns of all Nulls and/or NaNs
    null_counts = df.select([F.count(F.when(isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v > col_thresh]
    df = df.drop(*to_drop)
    
    #Remove Rows of all Nulls and/or NaNs
    df = df.na.drop(thresh=1)
    
    #Cast all values as double floats
    df = df.select([col(c).cast("double") for c in df.columns])
    
    #Impute all values 
    imputer = Imputer(
    inputCols=input_cols_wrangle, 
    outputCols=["{}_imputed".format(c) for c in df[input_cols_wrangle]]
    )
    df = imputer.fit(df).transform(df)
    
    #Vectorize all values
    vec_cols = df[imputer.getOutputCols()].columns
    assembler = VectorAssembler(
    inputCols = vec_cols,
    outputCol = "Features"
    )
    df = assembler.transform(df)
    
    #Scale all values
    scaler = MinMaxScaler(inputCol="Features", outputCol="scaledFeatures")
    scalerModel = scaler.fit(df)
    df = scalerModel.transform(df)
    
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
sdf_train_processed = process(sdf_train)
sdf_test_processed = process(sdf_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Revealed the impracticality of using the category columns due to too many nulls 

In [ ]:
input_cols_wrangle = sdf_train.columns[1:-1]


def process_cat(df):
#     # row_count for threshold below
#     col_thresh = df.count() - 1 
    
#     #Remove Columns of all Nulls and/or NaNs
#     null_counts = df.select([F.count(F.when(isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
#     to_drop = [k for k, v in null_counts.items() if v > col_thresh]
#     df = df.drop(*to_drop)
    
#     #Remove Rows of all Nulls and/or NaNs
#     df = df.na.drop(thresh=1)
    
    #Cast all values as double floats
    df = df.select([col(c).cast("double") for c in df.columns])
    
    #Impute all values 
    imputer = Imputer(
    inputCols=input_cols_wrangle, 
    outputCols=["{}_imputed".format(c) for c in df[input_cols_wrangle]]
    )
    df = imputer.fit(df).transform(df)
    
    #Vectorize all values
    vec_cols = df[imputer.getOutputCols()].columns
    assembler = VectorAssembler(
    inputCols = vec_cols,
    outputCol = "Features"
    )
    df = assembler.transform(df)
    
#     #Scale all values
#     scaler = MinMaxScaler(inputCol="Features", outputCol="scaledFeatures")
#     scalerModel = scaler.fit(df)
#     df = scalerModel.transform(df)
    
    return df

In [250]:
sdf_train.select("L0_S3_F69").show(1000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|L0_S3_F69|
+---------+
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|     null|
|   

In [233]:
# len(sdf_train_processed.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Vectorization Was A Success 

In [24]:
# sdf_train_processed.limit(1).toPandas()["Features"][0]

sdf_train_processed.select("Features", "scaledFeatures","Response").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------+
|            Features|      scaledFeatures|Response|
+--------------------+--------------------+--------+
|[-0.095,-0.138,-0...|[0.36731107205623...|     1.0|
|[0.00242157923201...|[0.53852650128649...|     1.0|
|[0.036,0.078,-0.1...|[0.59753954305799...|     1.0|
|[-0.003,-0.064,-0...|[0.52899824253075...|     1.0|
|[0.082,0.078,-0.0...|[0.67838312829525...|     1.0|
|[0.043,0.041,-0.0...|[0.60984182776801...|     1.0|
|[-0.029,-0.064,-0...|[0.48330404217926...|     1.0|
|[0.00242157923201...|[0.53852650128649...|     1.0|
|[0.00242157923201...|[0.53852650128649...|     1.0|
|[-0.036,-0.034,-0...|[0.47100175746924...|     1.0|
|[-0.016,-0.019,-0...|[0.50615114235500...|     1.0|
|[-0.121,-0.116,0....|[0.32161687170474...|     1.0|
|[-0.062,-0.026,-0...|[0.42530755711775...|     1.0|
|[0.049,0.026,-0.2...|[0.62038664323374...|     1.0|
|[-0.036,-0.064,0....|[0.47100175746924...|     1.0|
|[0.00242157923201...|[0.53852650128649...|   

# Ready For Machine Learning  

## Trying Logistic Regression First 

In [25]:
lr = LogisticRegression(featuresCol='scaledFeatures', labelCol='Response')

model = lr.fit(sdf_train_processed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
test_pred = model.transform(sdf_test_processed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
test_pred.limit(100).toPandas()[["Response","prediction"]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Response  prediction
0        1.0         1.0
1        1.0         1.0
2        0.0         1.0
3        0.0         1.0
4        0.0         1.0
..       ...         ...
95       0.0         1.0
96       0.0         1.0
97       0.0         1.0
98       0.0         1.0
99       0.0         1.0

[100 rows x 2 columns]

In [28]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol = "prediction",labelCol = "Response")
evaluator.evaluate(test_pred)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.4998926018584703

In [29]:
evaluator.getMetricName()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'areaUnderROC'

In [30]:
df_test = test_pred["Response","prediction"].toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
type(df_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>

In [32]:
df_test.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   Response  prediction
0       1.0         1.0
1       1.0         1.0
2       1.0         1.0
3       1.0         1.0
4       1.0         1.0

In [33]:
matthews_corrcoef(df_test["Response"], df_test["prediction"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-0.004359009802517331

In [34]:
accuracy_score(df_test["prediction"],df_test["Response"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.41159164460135333

In [35]:
average_precision_score(df_test["Response"], df_test["prediction"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.4115396309649478

## Trying Random Forest 

In [36]:
rf = RandomForestClassifier(featuresCol='scaledFeatures', labelCol='Response', numTrees=10)
model = rf.fit(sdf_train_processed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
predictions = model.transform(sdf_test_processed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol = "prediction",labelCol = "Response")
evaluator.evaluate(predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.5576880711778166

In [39]:
df_test2 = predictions["Response","prediction"].toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
matthews_corrcoef(df_test2["Response"], df_test2["prediction"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.1704262827799175

In [41]:
accuracy_score(df_test2["Response"], df_test2["prediction"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6200822078684675

# Trying SVM  

In [ ]:
model = SVMWithSGD.train(parsedData, iterations=100)

In [51]:
data = [
    LabeledPoint(0.0, [0.0]),
    LabeledPoint(1.0, [1.0]),
    LabeledPoint(1.0, [2.0]),
    LabeledPoint(1.0, [3.0])
]
svm = SVMWithSGD.train(sc.parallelize(data), iterations=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
svm = SVMWithSGD.train(sc.parallelize(sdf_train_processed["Features"].show()), iterations=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Column' object is not callable
Traceback (most recent call last):
TypeError: 'Column' object is not callable



In [81]:
sdf_train_processed.select("scaledFeatures").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|      scaledFeatures|
+--------------------+
|[0.68920521945432...|
|[0.68920521945432...|
|[0.67038285586961...|
|[0.82206405693950...|
|[0.67038285586961...|
|[0.65836298932384...|
|[0.67038285586961...|
|[0.67038285586961...|
|[0.73665480427046...|
|[0.67038285586961...|
|[0.56583629893238...|
|[0.65124555160142...|
|[0.81376037959667...|
|[0.67038285586961...|
|[0.62040332147093...|
|[0.75919335705812...|
|[0.79833926453143...|
|[0.59667852906287...|
|[0.67038285586961...|
|[0.75207591933570...|
+--------------------+
only showing top 20 rows

In [44]:
sdf_train_processed_2 = sdf_train_processed

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
sdf_train_processed_2 = sdf_train_processed_2[sdf_train_processed_2.columns[len(sdf_train_processed_2.columns)//2 - 1:]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
sdf_train_processed_2 = sdf_train_processed_2[sdf_train_processed_2.columns[::len(sdf_train_processed_2.columns)-1]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
sdf_train_processed_2 = sdf_train_processed_2[sdf_train_processed_2.columns[::-1]]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["Response", "scaledFeatures"],
    outputCol="features")

sdf_train_processed_3_vector = assembler.transform(sdf_train_processed_2).select("features")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
sdf_train_processed_3_vector = sdf_train_processed_3_vector.select("features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
from pyspark.sql.functions import *

def parsePoint(line):
    values = [float(x) for x in line]
    return LabeledPoint(values[0], values[1:])


sdf_train_labeled_point = sdf_train_processed_3_vector.rdd.map(lambda l: parsePoint(l[0]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
sdf_train_labeled_point.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[LabeledPoint(1.0, [0.36731107205623903,0.42172073342736244,0.2261410788381743,0.16926272066458983,0.5000000000000001,0.5297927461139896,0.4126984126984127,0.47058823529411764,0.3909866017052375,0.49446494464944657,0.17842741935483872,0.17842741935483872,0.3379549393414211,0.5372714486638537,0.19999999999999998,0.2545090180360721,0.2,0.211,0.8653846153846153,0.5,0.2,0.10932798395185556,0.3043478260869565,0.45083965330444203,0.3051690393925115,0.49602744673167215,0.39669634130615816,0.7638569880823401,0.5,0.6013001083423618,0.3296841581969485,0.21910501470360622,0.611318794258157,0.4022774679664552,0.6266996291718171,0.26895734597156395,0.32895093521279484,0.32499891569531036,0.3685969909878828,0.5,0.039000000000000035,0.16666666666666666,0.5,0.0,1.0,0.3083109919571046,0.6017505470459519,0.17460317460317462,0.4074766355140187,0.4444444444444444,0.47727272727272735,0.45940170940170943,0.3763440860215053,0.5771276595744681,0.44057971014492764,0.5,0.7586206896551724,0.3737704721267006,0.47

In [52]:
model = SVMWithSGD.train(sdf_train_labeled_point, iterations=100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
labelsAndPreds = sdf_train_labeled_point.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(sdf_train_labeled_point.count())
print("Training Error = " + str(trainErr))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training Error = 0.5941167731392066

In [54]:
pdf = labelsAndPreds.toDF().toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
pdf.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    _1  _2
0  1.0   1
1  1.0   1
2  1.0   1
3  1.0   1
4  1.0   1

In [56]:
matthews_corrcoef(pdf["_1"], pdf["_2"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-0.006645940621138985

In [57]:
accuracy_score(pdf["_1"], pdf["_2"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.40588322686079337

## Example of DF operation 

In [83]:
df = sqlContext.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+

### For quick querying purposes

In [66]:
sdf_train_processed.createOrReplaceTempView("parquetFile")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Major Class Imbalance 

In [71]:
spark.sql("SELECT Response,count(Response) as count FROM parquetFile group by Response").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+
|Response| count|
+--------+------+
|     0.0|941399|
|     1.0|  5479|
+--------+------+